In [11]:
import numpy as np

In [12]:
def read_obj(filename):
    mesh = {
        'vertices': [],
        'faces': [],
        'edges': []
    }
    with open(filename, 'r') as file:
        for line in file:
            if line.startswith('v '):
                mesh['vertices'].append([float(v) for v in line.strip().split()[1:]])
            elif line.startswith('f '):
                face = [int(idx.split('/')[0]) - 1 for idx in line.strip().split()[1:]]
                mesh['faces'].append(face)
                mesh['edges'].append([face[0], face[1]])
                mesh['edges'].append([face[1], face[2]])
                mesh['edges'].append([face[2], face[0]])
           
    mesh['vertices'] = np.array(mesh['vertices'])
    mesh['faces'] = np.array(mesh['faces'])
    mesh['edges'] = np.array(mesh['edges']) 
    return mesh

In [13]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from models.layers.mesh_pool import MeshPool
from models.layers.mesh_conv import MeshConv

In [14]:
def get_norm_args(norm_layer, nfeats_list):
    if hasattr(norm_layer, '__name__') and norm_layer.__name__ == 'NoNorm':
        norm_args = [{'fake': True} for f in nfeats_list]
    elif norm_layer.func.__name__ == 'GroupNorm':
        norm_args = [{'num_channels': f} for f in nfeats_list]
    elif norm_layer.func.__name__ == 'BatchNorm':
        norm_args = [{'num_features': f} for f in nfeats_list]
    else:
        raise NotImplementedError('normalization layer [%s] is not found' % norm_layer.func.__name__)
    return norm_args

In [15]:
class MeshConvNet(nn.Module):
    """Network for learning a global shape descriptor (classification)
    """
    def __init__(self, norm_layer, nf0, conv_res, nclasses, input_res, pool_res, fc_n,
                 nresblocks=3):
        super(MeshConvNet, self).__init__()
        self.k = [nf0] + conv_res
        self.res = [input_res] + pool_res
        norm_args = get_norm_args(norm_layer, self.k[1:])

        for i, ki in enumerate(self.k[:-1]):
            setattr(self, 'conv{}'.format(i), MResConv(ki, self.k[i + 1], nresblocks))
            setattr(self, 'norm{}'.format(i), norm_layer(**norm_args[i]))
            setattr(self, 'pool{}'.format(i), MeshPool(self.res[i + 1]))


        self.gp = torch.nn.AvgPool1d(self.res[-1])
        self.fc1 = nn.Linear(self.k[-1], fc_n)
        self.fc2 = nn.Linear(fc_n, nclasses)

    def forward(self, x, mesh):

        for i in range(len(self.k) - 1):
            x = getattr(self, 'conv{}'.format(i))(x, mesh)
            x = F.relu(getattr(self, 'norm{}'.format(i))(x))
            x = getattr(self, 'pool{}'.format(i))(x, mesh)

        x = self.gp(x)
        x = x.view(-1, self.k[-1])

        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

class MResConv(nn.Module):
    def __init__(self, in_channels, out_channels, skips=1):
        super(MResConv, self).__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.skips = skips
        self.conv0 = MeshConv(self.in_channels, self.out_channels, bias=False)
        for i in range(self.skips):
            setattr(self, 'bn{}'.format(i + 1), nn.BatchNorm2d(self.out_channels))
            setattr(self, 'conv{}'.format(i + 1),
                    MeshConv(self.out_channels, self.out_channels, bias=False))

    def forward(self, x, mesh):
        x = self.conv0(x, mesh)
        x1 = x
        for i in range(self.skips):
            x = getattr(self, 'bn{}'.format(i + 1))(F.relu(x))
            x = getattr(self, 'conv{}'.format(i + 1))(x, mesh)
        x += x1
        x = F.relu(x)
        return x

In [22]:
from torch.utils.data import DataLoader
from data.classification_data import ClassificationData  # Custom dataset
from data.base_dataset import collate_fn  # Custom collate function

class Options:
    def __init__(self):
        # Define all the necessary options with default values
        self.dataroot = "datasets/human_class"  # Dataset root directory
        self.phase = "train"  # Phase (train/test)
        self.batch_size = 8  # Batch size
        self.ninput_edges = 750  # Number of input edges for mesh
        self.num_threads = 4  # Number of workers for data loading
        self.gpu_ids = [0]  # Use GPU (set to [] for CPU)
        self.max_dataset_size = float('inf')  # Maximum dataset size
        self.serial_batches = False  # Shuffle data (False means shuffle)
        self.export_folder = None  # Folder for export results (if any)
        self.save_latest_freq = 1000  # Frequency of saving model
        self.print_freq = 100  # Frequency of printing log messages
        self.epoch_count = 1  # Start counting from which epoch
        self.niter = 50  # Number of iterations at base learning rate
        self.niter_decay = 50  # Number of iterations with learning rate decay
        self.save_epoch_freq = 10  # Frequency of saving model at each epoch
        self.verbose_plot = False  # Verbose plotting (e.g. weight plots)
        self.run_test_freq = 5  # Frequency to run tests
        self.num_aug = 20  # Number of augmentations  

# Initialize the Options object
opt = Options()

# Now we can access the options like opt.gpu_ids, opt.batch_size, etc.
print("Using GPU IDs:", opt.gpu_ids)
print("Batch size:", opt.batch_size)

# Then use `opt` just like you would in the original code

# Manually create dataset object using `opt`
dataset = ClassificationData(opt)

# Create DataLoader with all options specified in `opt`
dataloader = DataLoader(
    dataset,
    batch_size=opt.batch_size,
    shuffle=not opt.serial_batches,
    num_workers=opt.num_threads,
    collate_fn=collate_fn
)

# Example usage in training
for batch in dataloader:
    print(batch)
    break


Using GPU IDs: [0]
Batch size: 8
computing mean std from train data...


ValueError: [(0, 0), (0, -29250)] cannot contain negative values.